In [1]:
from graph_tool.all import *
import pandas as pd

In [2]:
g = Graph()

# Setup

In [3]:
import networkx as nx
import graph_tool as gt

def get_prop_type(value, key=None):
    """
    Performs typing and value conversion for the graph_tool PropertyMap class.
    If a key is provided, it also ensures the key is in a format that can be
    used with the PropertyMap. Returns a tuple, (type name, value, key)
    """
    if isinstance(key, str):
        # Encode the key as utf-8
        key = key.encode('utf-8', errors='replace')

    # Deal with the value
    if isinstance(value, bool):
        tname = 'bool'

    elif isinstance(value, int):
        tname = 'float'
        value = float(value)

    elif isinstance(value, float):
        tname = 'float'

    elif isinstance(value, str):
        tname = 'string'
        value = value.encode('utf-8', errors='replace')

    elif isinstance(value, dict):
        tname = 'object'

    else:
        tname = 'string'
        value = str(value)
        
    #If key is a byte value, decode it to string
    try:
        key = key.decode('utf-8')
    except AttributeError:
        pass

    return tname, value, key


def nx2gt(nxG):
    """
    Converts a networkx graph to a graph-tool graph.
    """
    # Phase 0: Create a directed or undirected graph-tool Graph
    gtG = gt.Graph(directed=nxG.is_directed())

    # Add the Graph properties as "internal properties"
    for key, value in list(nxG.graph.items()):
        # Convert the value and key into a type for graph-tool
        tname, value, key = get_prop_type(value, key)

        prop = gtG.new_graph_property(tname) # Create the PropertyMap
        
        gtG.graph_properties[key] = prop     # Set the PropertyMap
        gtG.graph_properties[key] = value    # Set the actual value

    # Phase 1: Add the vertex and edge property maps
    # Go through all nodes and edges and add seen properties
    # Add the node properties first
    nprops = set() # cache keys to only add properties once
    for node, data in nxG.nodes(data=True):

        # Go through all the properties if not seen and add them.
        for key, val in list(data.items()):            
            if key in nprops: continue # Skip properties already added

            # Convert the value and key into a type for graph-tool
            tname, _, key  = get_prop_type(val, key)

            prop = gtG.new_vertex_property(tname) # Create the PropertyMap
            gtG.vertex_properties[key] = prop     # Set the PropertyMap

            # Add the key to the already seen properties
            nprops.add(key)

    # Also add the node id: in NetworkX a node can be any hashable type, but
    # in graph-tool node are defined as indices. So we capture any strings
    # in a special PropertyMap called 'id' -- modify as needed!
    gtG.vertex_properties['id'] = gtG.new_vertex_property('string')

    # Add the edge properties second
    eprops = set() # cache keys to only add properties once
    for src, dst, data in nxG.edges(data=True):

        # Go through all the edge properties if not seen and add them.
        for key, val in list(data.items()):            
            if key in eprops: continue # Skip properties already added

            # Convert the value and key into a type for graph-tool
            tname, _, key = get_prop_type(val, key)
            
            prop = gtG.new_edge_property(tname) # Create the PropertyMap
            gtG.edge_properties[key] = prop     # Set the PropertyMap

            # Add the key to the already seen properties
            eprops.add(key)

    # Phase 2: Actually add all the nodes and vertices with their properties
    # Add the nodes
    vertices = {} # vertex mapping for tracking edges later
    for node, data in nxG.nodes(data=True):

        # Create the vertex and annotate for our edges later
        v = gtG.add_vertex()
        vertices[node] = v

        # Set the vertex properties, not forgetting the id property
        data['id'] = str(node)
        for key, value in list(data.items()):
            gtG.vp[key][v] = value # vp is short for vertex_properties

    # Add the edges
    for src, dst, data in nxG.edges(data=True):

        # Look up the vertex structs from our vertices mapping and add edge.
        e = gtG.add_edge(vertices[src], vertices[dst])

        # Add the edge properties
        for key, value in list(data.items()):
            gtG.ep[key][e] = value # ep is short for edge_properties

    # Done, finally!
    return gtG

# Load data

## Load nodes

In [103]:
nodes = pd.read_json('investigation/nodes.json')
nodes[:5]

,id,group,expanded,type,x,y,name,category
0,191129565003782,0.0,True,graph0_user,-208.402008,-2087.991699,NaN,NaN
1,65acb153c62cbe7e,NaN,True,graph0_device,-240.235794,-2049.601562,65acb153c62cbe7e,3.0
2,916c42063aa907ef,NaN,True,graph0_device,-219.403458,-2090.604248,916c42063aa907ef,3.0
3,605496a5d2507a6a,NaN,True,graph0_device,-145.893158,-2151.838379,605496a5d2507a6a,3.0
4,190802000002131,NaN,True,graph0_user,-217.167267,-2164.034424,190802000002131,0.0


## Load edges

In [104]:
links = pd.read_json('investigation/links.json')
links[:5]

,source,target
0,191129565003782,65acb153c62cbe7e
1,191129565003782,916c42063aa907ef
2,191129565003782,605496a5d2507a6a
3,190802000002131,916c42063aa907ef
4,190802000009180,605496a5d2507a6a


## Make Graph

In [107]:
G = nx.Graph()

In [108]:
G.add_nodes_from(nodes['id'].tolist())

In [109]:
G.add_edges_from([tuple(x) for x in links.to_numpy()])

In [110]:
print("Nodes: ",  G.number_of_nodes())
print("Edges: ", G.number_of_edges())

Nodes:  79073
Edges:  62845


# Convert to graph-tool

In [111]:
gtG = nx2gt(G)

In [112]:
gtG

<Graph object, undirected, with 79073 vertices and 62845 edges, 1 internal vertex property, at 0x7f79bfcf4d60>

# Graph Analysis

## Betweenness Metrics

In [113]:
vp, ep = gt.centrality.betweenness(gtG)

In [114]:
gtG.vp.id[0], vp[1]

('191129565003782', 8.052843197889652e-06)

In [138]:
betweenness_list = list({"id": x[0], "s": x[1]} for x in zip(gtG.vp.id, vp))

In [139]:
betweenness_list = sorted(betweenness_list, key=lambda k: k['s'], reverse=True)
betweenness_list[:30]

[{'id': '3a43b1c0-3d90-4d67-9f12-2143a53a8c8c', 's': 0.15098124796355988},
 {'id': '181020000005167', 's': 0.11161104275390461},
 {'id': '21e31aedd17e0e83', 's': 0.10849944246527257},
 {'id': 'bb0c21049fabcd30', 's': 0.10087991301091301},
 {'id': '170925000022096', 's': 0.0852634613998933},
 {'id': '357b1146696777de', 's': 0.08300013257843425},
 {'id': '190927000013903', 's': 0.08099509138306299},
 {'id': 'ea313a5dc143332d', 's': 0.08023763030448713},
 {'id': '2016d85ea3619630', 's': 0.07945175943651821},
 {'id': '190518000012062', 's': 0.07928312736806242},
 {'id': '191013221000689', 's': 0.07836227177228314},
 {'id': '191011798006758', 's': 0.07800985113624156},
 {'id': '36bee595987312e2', 's': 0.07800570953782641},
 {'id': '3c5ec67cc28e1769', 's': 0.07535392116457262},
 {'id': '190924000011186', 's': 0.07152930664418132},
 {'id': '190221000014512', 's': 0.0700257228870209},
 {'id': '191006682003225', 's': 0.06796817705029755},
 {'id': '191012000009351', 's': 0.06781497643635227},
 {

In [140]:
print(gt.centrality.central_point_dominance(gtG, vp))

0.15083835105444052


In [141]:
for x in betweenness_list:
    if x['id'] == 'c650e49f-a0da-4c4c-a3ad-e1957a4d891f':
        print(x)
        break

{'id': 'c650e49f-a0da-4c4c-a3ad-e1957a4d891f', 's': 0.03760078188620966}


## PageRank

In [119]:
pr = gt.centrality.pagerank(gtG)

In [126]:
pagerank_list = list({"id": x[0], "s": x[1]} for x in zip(gtG.vp.id, pr))

In [127]:
sorted(pagerank_list, key=lambda k: k['s'], reverse=True)[:30]

[{'id': '181104000001278', 's': 0.015281048474485774},
 {'id': '92ff9530442ba3af', 's': 0.007399437740596529},
 {'id': 'ea313a5dc143332d', 's': 0.007361123111820632},
 {'id': '170223000002289', 's': 0.006275777042987438},
 {'id': '4fc9899a2795a0ce', 's': 0.006182504930644087},
 {'id': '06db97b336d9dfda', 's': 0.004368357617363605},
 {'id': '547e7aa35af55fb7', 's': 0.004265969037056791},
 {'id': '1fa68cdc3d4067a5', 's': 0.003714108018976234},
 {'id': 'fdd18fb79098f389', 's': 0.003704151191527766},
 {'id': '0af4a07c22dff7e3', 's': 0.0034105941357226483},
 {'id': '190222000001425', 's': 0.0033866678664276153},
 {'id': '190111000003775', 's': 0.0031042983661353574},
 {'id': '180002c1e01fab36', 's': 0.0025168834915975332},
 {'id': '190002c1e01fab87', 's': 0.0024991517769794454},
 {'id': '1309c23d9e80c742', 's': 0.0022017536746402116},
 {'id': '191011000013977', 's': 0.001908760574237535},
 {'id': '170925000026056', 's': 0.001890771778486877},
 {'id': 'dd4758ae4fa3f225', 's': 0.0017850737652

In [145]:
clust = gt.clustering.local_clustering(gtG)

In [150]:
cluster = list({"id": x[0], "group": x[1]} for x in zip(gtG.vp.id, clust))

In [154]:
cluster_df = pd.DataFrame(cluster)
cluster_df

,id,group
0,191129565003782,0.0
1,65acb153c62cbe7e,0.0
2,916c42063aa907ef,0.0
3,605496a5d2507a6a,0.0
4,190802000002131,0.0
...,...,...
79068,200806000167857,0.0
79069,200819000010097,0.0
79070,200819000029258,0.0
79071,200921000001529,0.0


In [155]:
cluster_df.group.value_counts()

0.0    79073
Name: group, dtype: int64

In [205]:
state = gt.inference.minimize_blockmodel_dl(gtG)

In [ ]:
state.draw(vertex_shape=state.get_blocks(),
         output="polbooks_blocks_mdl.svg")

In [ ]:
state.get_B()

# Graph0

## Kruskal

In [57]:
userID_accountID = pd.read_csv('raw/userID_accountID.csv')
userID_cardID = pd.read_csv('raw/userID_cardID.csv')
userID_deviceID = pd.read_csv('raw/userid_deviceid.csv')

In [58]:
userID_accountID['userID'] = userID_accountID['userID'].apply(lambda x: f'user/{x}')
userID_accountID['accountID'] = userID_accountID['accountID'].apply(lambda x: f'account/{x}')
userID_accountID.head()

,userID,accountID
0,user/200221000026960,account/200221100002264
1,user/191206720001042,account/191206100000350
2,user/191210484000661,account/191210100000238
3,user/190709000001059,account/190707100001431
4,user/191227000009403,account/191227100000397


In [59]:
userID_cardID['userID'] = userID_cardID['userID'].apply(lambda x: f'user/{x}')
userID_cardID['cardID'] = userID_cardID['cardID'].apply(lambda x: f'card/{x}')
userID_cardID.head()

,userID,cardID
0,user/191111735008352,card/191111000016864
1,user/180818000008076,card/200524000004399
2,user/190120000001593,card/190612000556786
3,user/191124227003671,card/191124000007604
4,user/191012537001139,card/191012000000490


In [60]:
userID_deviceID['userID'] = userID_deviceID['userID'].apply(lambda x: f'user/{x}')
userID_deviceID['deviceID'] = userID_cardID['deviceID'].apply(lambda x: f'device/{x}')
userID_deviceID.head()

,userID,deviceID
0,user/190215000014037,device/5e34bdd5266ff7c6
1,user/190313000016148,device/1761c1b0d746b8e8
2,user/190318000000574,device/14e0d55eb848e527
3,user/190722000001381,device/1f6b129e317e35fa
4,user/191016219002578,device/0e2fa6b350f0f020


### Sub-processing

In [35]:
subuser_list = userID_deviceID['userID'].tolist()[:5000]

In [36]:
subuserIDdeviceID = userID_deviceID.loc[userID_deviceID.apply(lambda x: x['userID'] in subuser_list, axis=1)]

In [61]:
subuserIDdeviceID

,userID,deviceID
0,190215000014037,5e34bdd5266ff7c6
1,190313000016148,1761c1b0d746b8e8
2,190318000000574,14e0d55eb848e527
3,190722000001381,1f6b129e317e35fa
4,191016219002578,0e2fa6b350f0f020
...,...,...
218339,190905000002054,3c00559c3e3d8259
218340,160827000000530,b4f5eed0-b0a1-4a1b-8500-d5f3ad3ea8c0
218358,190301000002445,b99d39a3-2e45-4212-b453-52bfeaa0cda5
218361,190829000000083,bbe1dca0e9fca6b6


### Total-processing

In [66]:
user_list = list(set(userID_accountID['userID'].tolist() + 
            userID_deviceID['userID'].tolist() +
            userID_cardID['userID'].tolist()))

In [68]:
account_list = list(set(userID_accountID['accountID'].tolist()))

In [69]:
card_list = list(set(userID_cardID['cardID'].tolist()))

In [70]:
device_list = list(set(userID_deviceID['deviceID'].tolist()))

## Make Graph

In [72]:
nodes = user_list + account_list + card_list + device_list

In [79]:
links = [tuple(x) for x in userID_accountID.to_numpy()] + [tuple(x) for x in userID_cardID.to_numpy()]+ [tuple(x) for x in userID_deviceID.to_numpy()]

In [85]:
G = nx.Graph()

In [86]:
G.add_nodes_from(nodes)

In [87]:
G.add_edges_from(links)

In [88]:
print("Nodes: ",  G.number_of_nodes())
print("Edges: ", G.number_of_edges())

Nodes:  311036
Edges:  280277


In [89]:
gtG = nx2gt(G)

In [90]:
gtG

<Graph object, undirected, with 311036 vertices and 280277 edges, 1 internal vertex property, at 0x7f51e2f529a0>

In [91]:
mst = graph_tool.topology.min_spanning_tree(gtG)

In [92]:
mst.get_array()

PropertyArray([1, 0, 1, ..., 1, 1, 1], dtype=uint8)

In [93]:
u = graph_tool.GraphView(gtG, efilt=mst)

In [48]:
graph_tool.draw.graph_draw(u, output="mst.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f51d7e86130, at 0x7f51e350c8e0>

## Process

In [97]:
u.get_vertices()

array([     0,      1,      2, ..., 311033, 311034, 311035])

In [98]:
u.get_edges()

array([[     0, 169389],
       [     0, 201941],
       [     1, 170804],
       ...,
       [130410, 308135],
       [130410, 234145],
       [130410, 205225]])

In [ ]:
u